In [1]:
#For this real world use case we tackle a problem from the field of Natural Language Processing (NLP). 
#The task is to classify movie reviews into classes expressing positive sentiment about a movie or negative sentiment. 
#To perform a task like this, the model must be able to understand natural language, that is it must know 
#the meaning of an entire sentence as expressed by its class prediction. 
#Recurrent Neural Networks (RNNs) are usually well suited for tasks involving sequential data like sentences however, 
#we would apply a 1-dimensional Convolutional Neural Network (CNN) model to this task as it is easier to train 
#and produces comparable results.

#The dataset we would use is the IMDB sentiment database which contains 25,000 movie reviews in the training set 
#and 25,000 reviews in the test set. TFLearn bundles this dataset alongside others so we would access it from the datasets module.

#First we import the IMDB sentiment dataset module and other relevant components from TFLearn such as convolutional layers, 
#fully connected layers, data utilities etc.

import tensorflow as tf

import tflearn

from tflearn.layers.core import input_data, dropout, fully_connected

from tflearn.layers.conv import conv_1d, global_max_pool

from tflearn.layers.merge_ops import merge

from tflearn.layers.estimator import regression

from tflearn.data_utils import to_categorical, pad_sequences

from tflearn.datasets import imdb

C:\Users\RUIZ\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#The next step is to actually load the dataset into the train and test splits

# load IMDB dataset

train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000, valid_portion=0.1)

trainX, trainY = train

testX, testY = test

In [3]:
#The next phase involves preprocessing the data where we pad sequences which means we set a maximum sentence length 
#and for sentences less than the maximum sentence length we add zeros to them. The reason is to make sure that all sentences 
#are of the same length before they are passed to the neural network model. The labels in the train and test sets 
#are also converted to categorical values.

# data preprocessing

# sequence padding

trainX = pad_sequences(trainX, maxlen=100, value=0.)

testX = pad_sequences(testX, maxlen=100, value=0.)

# converting labels to binary vectors

trainY = to_categorical(trainY, nb_classes=2)

testY = to_categorical(testY, nb_classes=2)

In [4]:
#the next step is to describe a 1-dimensional Convolutional Neural  Network model using the building blocks provided to us by TFLearn.

# building the convolutional network

#The network contains 3 1-dimensional convolutional layers, a global max pooling layer used to reduce 
#the dimension of convolutions, a dropout layer used for regularization and a fully connected layer.

network = input_data(shape=[None, 100], name='input')

network = tflearn.embedding(network, input_dim=10000, output_dim=128)

branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")

branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")

branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")

network = merge([branch1, branch2, branch3], mode='concat', axis=1)

network = tf.expand_dims(network, 2)

network = global_max_pool(network)

network = dropout(network, 0.5)

network = fully_connected(network, 2, activation='softmax')

network = regression(network, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='target')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
#Finally, we declare the model and call fit method on it to begin training.

# training

#The trained model achieves an accuracy of 0.7992 on the test set which is to say it correctly classified 
#the sentiment expressed in 79% of sentences.

model = tflearn.DNN(network, tensorboard_verbose=0)

model.fit(trainX, trainY, n_epoch=5, shuffle=True, validation_set=(testX, testY), show_metric=True, batch_size=32)

Training Step: 3519  | total loss: 0.09039 | time: 159.175s
| Adam | epoch: 005 | loss: 0.09039 - acc: 0.9656 -- iter: 22496/22500
Training Step: 3520  | total loss: 0.10540 | time: 163.064s
| Adam | epoch: 005 | loss: 0.10540 - acc: 0.9628 | val_loss: 0.56382 - val_acc: 0.7992 -- iter: 22500/22500
--
